<a href="https://colab.research.google.com/github/gulabpatel/LinearProgramming/blob/main/02%3A%20Pyomo_glpk_binary_knapshak_maxmize_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Video help : https://www.youtube.com/watch?v=pxCogCylmKs

\begin{align}
    \text{max}~~  & 3 x_1 + 4 x_2 + 5 x_3 + 8 x_4 + 9 x_5 \\
    \text{s.t.}~~ & 2 x_1 + 3 x_2 + 4 x_3 + 7 x_4 + 9 x_5 \le 20 \\
                  & x_1, x_2, x_3, x_4, x_5 \in \{0, 1\}
\end{align}
    

In [1]:
!pip install pyomo
!apt install glpk-utils
# !pip install glpk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
glpk-utils is already the newest version (4.65-1).
The following packages were automatically installed and are no longer required:
  libnvidia-common-460 nsight-compute-2020.2.0
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [2]:
import pyomo.environ as pe
import pyomo.opt as po

In [3]:
solver = po.SolverFactory('glpk') #GNU Linear Programming Kit

In [4]:
model = pe.ConcreteModel()

model.x1 = pe.Var(domain=pe.Binary)
model.x2 = pe.Var(domain=pe.Binary)
model.x3 = pe.Var(domain=pe.Binary)
model.x4 = pe.Var(domain=pe.Binary)
model.x5 = pe.Var(domain=pe.Binary)

In [5]:
obj_expr = 3*model.x1 + 4*model.x2 + 5*model.x3 + 8*model.x4 + 9*model.x5
model.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [6]:
con_expr = 2*model.x1 + 3*model.x2 + 4*model.x3 + 5*model.x4 + 9*model.x5 <= 20
model.con = pe.Constraint(expr=con_expr)

Solve and Postprocess

In [7]:
result = solver.solve(model, tee=True)

GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write /tmp/tmp7wn23hch.glpk.raw --wglp /tmp/tmp48sljg3o.glpk.glp --cpxlp
 /tmp/tmpqy2eeepc.pyomo.lp
Reading problem data from '/tmp/tmpqy2eeepc.pyomo.lp'...
/tmp/tmpqy2eeepc.pyomo.lp:31: warning: lower bound of variable 'x1' redefined
/tmp/tmpqy2eeepc.pyomo.lp:31: warning: upper bound of variable 'x1' redefined
2 rows, 6 columns, 6 non-zeros
5 integer variables, all of which are binary
36 lines were read
Writing problem data to '/tmp/tmp48sljg3o.glpk.glp'...
25 lines were written
GLPK Integer Optimizer, v4.65
2 rows, 6 columns, 6 non-zeros
5 integer variables, all of which are binary
Preprocessing...
3 constraint coefficient(s) were reduced
1 row, 5 columns, 5 non-zeros
5 integer variables, all of which are binary
Scaling...
 A: min|aij| =  2.000e+00  max|aij| =  3.000e+00  ratio =  1.500e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 1
Solving LP relaxatio

In [8]:
print(pe.value(model.x1))
print(pe.value(model.x2))
print(pe.value(model.x3))
print(pe.value(model.x4))
print(pe.value(model.x5))
print(pe.value(model.obj))

1.0
0.0
1.0
1.0
1.0
25.0


## Model (General)
The best way to use Pyomo is to implement the general form of the problem. The instance above is generalized by the formulation below. The set of items is $N$. The coefficients $c_i$ and $a_i$ are the cost and size, respectively of item $i$.

\begin{align}
    \text{max}~~  & \sum_{i \in N}{c_i x_i} \\
    \text{s.t.}~~ & \sum_{i \in N}{a_i x_i} \le b \\
                  & x_i \in \{0, 1\}, \forall i \in N
\end{align}

## Implement (General)

In [9]:
model = pe.ConcreteModel()

In [10]:
model.N = pe.RangeSet(1, 5)

In [11]:
print(set(model.N))

{1, 2, 3, 4, 5}


In [12]:
c = {1: 3, 2: 4, 3: 5, 4: 8, 5: 9}
a = {1: 2, 2: 3, 3: 4, 4: 5, 5: 9}
b = 20

In [13]:
model.c = pe.Param(model.N, initialize=c)
model.a = pe.Param(model.N, initialize=a)
model.b = pe.Param(initialize=b)

For most model components, `print(model.component)` will print the name of the component (a string) and not the value (in most cases, a number).

In [14]:
print(model.c)
print(model.c[2])
print(model.b)
print(model.b.value)
print(pe.value(model.b))

c
4
b
20
20


In [15]:
model.x = pe.Var(model.N, domain=pe.Binary)

In [16]:
obj_expr = sum(model.c[i] * model.x[i] for i in model.N)
model.obj = pe.Objective(sense=pe.maximize, expr=obj_expr)

In [17]:
con_lhs_expr = sum(model.a[i] * model.x[i] for i in model.N)
con_rhs_expr = model.b
model.con = pe.Constraint(expr=(con_lhs_expr <= con_rhs_expr))

In [18]:
result = solver.solve(model)

After solving, variable values may be accessed either by `pe.value(model.myvar)` or `model.myvar.value`.

In [19]:
for i in model.N:
    print(pe.value(model.x[i]))
print(pe.value(model.obj))

1.0
0.0
1.0
1.0
1.0
25.0
